# ネットワークを遺伝子単位で出力する


In [109]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI
/mnt/e/Research/TSUMUGI


In [110]:
P = print
from pprint import pprint as PP
from collections import Counter as C
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import json
import pickle
import networkx as nx
import shutil
import gzip

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [111]:
overlap = pickle.load(open("data/overlap/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
df_overlap  # 133281 rows × 5 columns

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,0610010K14Rik,4933427D14Rik,0.500,3,[embryonic lethality prior to organogenesis (H...
1,0610010K14Rik,Acvr1,0.600,3,[embryonic lethality prior to organogenesis (H...
2,0610010K14Rik,Adss2,0.375,3,[embryonic lethality prior to organogenesis (H...
3,0610010K14Rik,Ahcy,0.500,3,[embryonic lethality prior to organogenesis (H...
4,0610010K14Rik,Anapc4,0.750,3,[embryonic lethality prior to organogenesis (H...
...,...,...,...,...,...
133276,Zfp503,Zwilch,0.208,5,"[abnormal embryo size (Homo), abnormal placent..."
133277,Zfp512,Zpld1,0.429,3,"[abnormal behavior (Homo), decreased thigmotax..."
133278,Zfp612,Zfp641,0.500,3,"[decreased mean corpuscular volume (Homo), hyp..."
133279,Zfp638,Znhit2,0.429,3,[embryonic lethality prior to organogenesis (H...


In [112]:
marker_mp = json.load(open("data/annotation/symbol_mptermname.json"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term_name"])
marker_mp  # 7746 rows × 2 columns
# DFをdictに変換
marker_mp_dict = dict(zip(marker_mp.marker_symbol, marker_mp.mp_term_name))

## ひとつの遺伝子をハンドルする


In [113]:
gene_symbol = "Rab10"

In [114]:
P(sum(df_overlap.marker1 == gene_symbol))
P(sum(df_overlap.marker2 == gene_symbol))

13
29


In [115]:
import networkx as nx

In [116]:
G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")

In [117]:
# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)

P(subgraph)
P(len(subgraph.nodes))

Graph with 43 nodes and 365 edges
43


In [118]:
df_overlap

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,0610010K14Rik,4933427D14Rik,0.500,3,[embryonic lethality prior to organogenesis (H...
1,0610010K14Rik,Acvr1,0.600,3,[embryonic lethality prior to organogenesis (H...
2,0610010K14Rik,Adss2,0.375,3,[embryonic lethality prior to organogenesis (H...
3,0610010K14Rik,Ahcy,0.500,3,[embryonic lethality prior to organogenesis (H...
4,0610010K14Rik,Anapc4,0.750,3,[embryonic lethality prior to organogenesis (H...
...,...,...,...,...,...
133276,Zfp503,Zwilch,0.208,5,"[abnormal embryo size (Homo), abnormal placent..."
133277,Zfp512,Zpld1,0.429,3,"[abnormal behavior (Homo), decreased thigmotax..."
133278,Zfp612,Zfp641,0.500,3,"[decreased mean corpuscular volume (Homo), hyp..."
133279,Zfp638,Znhit2,0.429,3,[embryonic lethality prior to organogenesis (H...


In [119]:
"""
ノードが多すぎるとWebページが描画できない問題を回避するため、
ノード数を500以下にするために最適なoverlap_ratioを求める
"""
target_columns = 500
# 二分探索の範囲
low, high = df_overlap["overlap_ratio"].min(), df_overlap["overlap_ratio"].max()
best_overlap_ratio = None

while low <= high:
    mid = (low + high) / 2

    # overlap_ratio >= mid のデータをフィルタリング
    df_mid = df_overlap[df_overlap["overlap_ratio"] >= mid]

    G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
    # ノードAと直接つながっているノードのみを取得
    try:
        neighbors = list(G.neighbors(gene_symbol))
    except:
        high = mid - 1e-6
        continue        
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)

    row_count = len(subgraph.nodes)
    # ターゲット列数に近い場合、結果を保存
    if row_count == target_columns:
        best_overlap_ratio = mid
        break
    elif row_count > target_columns:
        # 列数が多い場合、範囲を上げる
        best_overlap_ratio = mid
        low = mid + 1e-6
    else:
        # 列数が少ない場合、範囲を下げる
        best_overlap_ratio = mid
        high = mid - 1e-6
    print(low, high, mid, best_overlap_ratio, row_count)

df_overlap = df_overlap[df_overlap["overlap_ratio"] >= best_overlap_ratio]


0.026 0.26949850000000003 0.2694995 0.2694995 26
0.026 0.14774825000000003 0.14774925000000003 0.14774925000000003 43
0.026 0.08687312500000001 0.08687412500000001 0.08687412500000001 43
0.026 0.0564355625 0.0564365625 0.0564365625 43
0.026 0.04121678125 0.04121778125 0.04121778125 43
0.026 0.033607390625 0.033608390625 0.033608390625 43
0.026 0.029802695312499995 0.029803695312499996 0.029803695312499996 43
0.026 0.027900347656249998 0.02790134765625 0.02790134765625 43
0.026 0.026949173828125 0.026950173828125 0.026950173828125 43
0.026 0.0264735869140625 0.0264745869140625 0.0264745869140625 43
0.026 0.026235793457031246 0.026236793457031247 0.026236793457031247 43
0.026 0.026116896728515623 0.026117896728515624 0.026117896728515624 43
0.026 0.02605744836425781 0.02605844836425781 0.02605844836425781 43
0.026 0.026027724182128904 0.026028724182128905 0.026028724182128905 43
0.026 0.026012862091064452 0.026013862091064453 0.026013862091064453 43
0.026 0.026005431045532226 0.026006431

In [120]:
P(subgraph.nodes())

['Hars2', 'Prpf39', 'Wee1', 'Ebna1bp2', 'Med10', 'Rnaseh2a', 'Mccc2', 'Osbp', 'Srprb', 'Atp6v1b2', 'Sptb', 'Atl2', 'Tomm40', 'Ddx46', 'Haus4', 'Thoc3', 'Med28', 'Btf3', 'Trappc11', 'Asxl1', 'Chp1', 'Cct3', 'U2af2', 'Tomm22', 'Ccdc59', 'Gtpbp4', 'Dnm2', 'Scfd1', 'Clptm1', 'Ints8', 'Med25', 'Mcmbp', 'Fbxo5', 'Dhx15', 'Cdca5', 'Ppie', 'Atp2b1', 'Ints3', 'Tfrc', 'Rab10', 'U2af1', 'Lmbrd1', 'Rpain']


In [121]:
# nodesを用意

node_json = []
for node in subgraph.nodes():
    annotation = marker_mp_dict[node]
    node_color = 1 if node == gene_symbol else 0
    node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

P(node_json[0])

{'data': {'id': 'Hars2', 'label': 'Hars2', 'node_color': 0, 'annotation': ['abnormal auditory brainstem response (Hetero)', 'decreased grip strength (Hetero)', 'decreased grip strength (Hetero, ♀)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}


In [122]:
# edgesを用意
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

edge_info = pd.DataFrame({"id": ["-".join(sorted([a, b])) for a, b in subgraph.edges()]}).merge(df_overlap, on="id")
edge_json = []
for edge in edge_info.itertuples():
    edge_json.append(
        {
            "data": {
                "source": edge.marker1,
                "target": edge.marker2,
                "edge_size": edge.overlap_ratio,
                "annotation": edge.overlapped_mp,
            }
        }
    )
P(edge_json[:3])

[{'data': {'source': 'Atp2b1', 'target': 'Hars2', 'edge_size': 0.3, 'annotation': ['abnormal auditory brainstem response (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}, {'data': {'source': 'Cdca5', 'target': 'Hars2', 'edge_size': 0.3, 'annotation': ['abnormal auditory brainstem response (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}, {'data': {'source': 'Hars2', 'target': 'Haus4', 'edge_size': 0.6, 'annotation': ['abnormal auditory brainstem response (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}]


/tmp/ipykernel_995575/1419414854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)


In [123]:
# Cytoscape.js用のJSON形式に変換
cytoscape_data = node_json + edge_json

In [124]:
# JSONに変換
cytoscape_json = json.dumps(cytoscape_data, indent=2)
print(cytoscape_json[:3])

[
 


In [125]:
# Path("Rab10.json").write_text(cytoscape_json)

## 一括処理


In [126]:
overlap = pickle.load(open("data/overlap/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
P(len(df_overlap))  # 133281 rows × 5 columns

133281


In [127]:
gene_symbols = df_overlap.marker1.unique().tolist()
gene_symbols += df_overlap.marker2.unique().tolist()
gene_symbols = list(set(gene_symbols))
gene_symbols.sort()  # 以下のfor文で、どこまで遺伝子が処理されたのか途中経過を見積もるためのソート
P(gene_symbols[:3])
P(len(gene_symbols)) # 6003

['0610010K14Rik', '0610040J01Rik', '1500009L16Rik']
4139


In [128]:
output_dir = Path("data", "network", "gene_symbol")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [129]:
for gene_symbol in gene_symbols:
    """
    ノードが多すぎるとWebページが描画できない問題を回避するため、
    ノード数を200以下にするために最適なoverlap_ratioを求める
    """
    # 今の処理
    df_filtered = df_overlap[
            (df_overlap['marker1'] == gene_symbol) |
            (df_overlap['marker2'] == gene_symbol)
        ]

    G = nx.from_pandas_edgelist(df_filtered, "marker1", "marker2")

    # ノードAと直接つながっているノードのみを取得
    neighbors = list(G.neighbors(gene_symbol))
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)

    target_number_of_nodes = 200
    if len(subgraph.nodes) > target_number_of_nodes:
        # 二分探索の範囲
        low, high = df_filtered["overlap_ratio"].min(), df_filtered["overlap_ratio"].max()
        best_overlap_ratio = None

        while low <= high:
            mid = (low + high) / 2

            # overlap_ratio >= mid のデータをフィルタリング
            df_mid = df_filtered[df_filtered["overlap_ratio"] >= mid]

            G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
            # ノードAと直接つながっているノードのみを取得
            try:
                neighbors = list(G.neighbors(gene_symbol))
            except:
                high = mid - 1e-6
                continue        
            subgraph_nodes = [gene_symbol] + neighbors
            subgraph = G.subgraph(subgraph_nodes)

            row_count = len(subgraph.nodes)
            # ターゲット列数に近い場合、結果を保存
            if target_number_of_nodes - 25 < row_count < target_number_of_nodes + 25:
                best_overlap_ratio = mid
                break
            elif row_count > target_number_of_nodes:
                # 列数が多い場合、範囲を上げる
                best_overlap_ratio = mid
                low = mid + 1e-6
            else:
                # 列数が少ない場合、範囲を下げる
                best_overlap_ratio = mid
                high = mid - 1e-6

        df_nodes = df_filtered[df_filtered["overlap_ratio"] >= best_overlap_ratio]
        G = nx.from_pandas_edgelist(df_nodes, "marker1", "marker2")
        # ノードAと直接つながっているノードのみを取得
        neighbors = list(G.neighbors(gene_symbol))
        subgraph_nodes = [gene_symbol] + neighbors
        subgraph = G.subgraph(subgraph_nodes)

    # nodesを用意
    node_json = []
    for node in subgraph.nodes():
        annotation = marker_mp_dict[node]
        node_color = 1 if node == gene_symbol else 0
        node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

    # edgesを用意
    df_edge = df_overlap[
            (df_overlap['marker1'].isin(subgraph.nodes())) &
            (df_overlap['marker2'].isin(subgraph.nodes()))
        ]

    edge_json = []
    for edge in df_edge.itertuples():
        edge_json.append(
            {
                "data": {
                    "source": edge.marker1,
                    "target": edge.marker2,
                    "edge_size": edge.overlap_ratio,
                    "annotation": edge.overlapped_mp,
                }
            }
        )
    network_json = node_json + edge_json

    # Output as JSON
    if network_json:
        output_json = output_dir / f"{gene_symbol}.json.gz"
        with gzip.open(output_json, "wt", encoding="utf-8") as f:
            json.dump(network_json, f, indent=4)

# 4m

In [130]:
%%bash
ls -lhS data/network/gene_symbol/ | head -n 5 # total 4.6G -> 133M → 975M
# total 118M
# -rwxrwxrwx 1 kuno kuno 198K Jan 25 10:31 Fam161b.json.gz
# -rwxrwxrwx 1 kuno kuno 198K Jan 25 10:32 Pabir2.json.gz
# -rwxrwxrwx 1 kuno kuno 185K Jan 25 10:31 Lrrc17.json.gz
# -rwxrwxrwx 1 kuno kuno 182K Jan 25 10:30 Cer1.json.gz

total 118M
-rwxrwxrwx 1 kuno kuno 198K Jan 25 10:31 Fam161b.json.gz
-rwxrwxrwx 1 kuno kuno 198K Jan 25 10:32 Pabir2.json.gz
-rwxrwxrwx 1 kuno kuno 185K Jan 25 10:31 Lrrc17.json.gz
-rwxrwxrwx 1 kuno kuno 182K Jan 25 10:30 Cer1.json.gz


In [131]:
Path("data/overlap/available_gene_symbols.txt").write_text("\n".join(gene_symbols) + "\n")
print(len(gene_symbols)) # 4416 -> 4244 → 6003

4139


# Debug



## 一つの遺伝子のみ取り扱う

In [132]:
overlap = pickle.load(open("data/overlap/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
df_overlap  # 133281 rows × 5 columns

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,0610010K14Rik,4933427D14Rik,0.500,3,[embryonic lethality prior to organogenesis (H...
1,0610010K14Rik,Acvr1,0.600,3,[embryonic lethality prior to organogenesis (H...
2,0610010K14Rik,Adss2,0.375,3,[embryonic lethality prior to organogenesis (H...
3,0610010K14Rik,Ahcy,0.500,3,[embryonic lethality prior to organogenesis (H...
4,0610010K14Rik,Anapc4,0.750,3,[embryonic lethality prior to organogenesis (H...
...,...,...,...,...,...
133276,Zfp503,Zwilch,0.208,5,"[abnormal embryo size (Homo), abnormal placent..."
133277,Zfp512,Zpld1,0.429,3,"[abnormal behavior (Homo), decreased thigmotax..."
133278,Zfp612,Zfp641,0.500,3,"[decreased mean corpuscular volume (Homo), hyp..."
133279,Zfp638,Znhit2,0.429,3,[embryonic lethality prior to organogenesis (H...


In [133]:
marker_mp = json.load(open("data/annotation/symbol_mptermname.json"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term_name"])
marker_mp  # 7746 rows × 2 columns
# DFをdictに変換
marker_mp_dict = dict(zip(marker_mp.marker_symbol, marker_mp.mp_term_name))

In [134]:
gene_symbol = "Rhd"

In [135]:
# 今の処理
df_filtered = df_overlap[
        (df_overlap['marker1'] == gene_symbol) |
        (df_overlap['marker2'] == gene_symbol)
    ]

G = nx.from_pandas_edgelist(df_filtered, "marker1", "marker2")

# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)
if len(subgraph.nodes) > 500:
    target_columns = 500
    # 二分探索の範囲
    low, high = df_filtered["overlap_ratio"].min(), df_filtered["overlap_ratio"].max()
    best_overlap_ratio = None

    while low <= high:
        mid = (low + high) / 2

        # overlap_ratio >= mid のデータをフィルタリング
        df_mid = df_filtered[df_filtered["overlap_ratio"] >= mid]

        G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
        # ノードAと直接つながっているノードのみを取得
        try:
            neighbors = list(G.neighbors(gene_symbol))
        except:
            high = mid - 1e-6
            continue        
        subgraph_nodes = [gene_symbol] + neighbors
        subgraph = G.subgraph(subgraph_nodes)

        row_count = len(subgraph.nodes)
        # ターゲット列数に近い場合、結果を保存
        if row_count == target_columns:
            best_overlap_ratio = mid
            break
        elif row_count > target_columns:
            # 列数が多い場合、範囲を上げる
            best_overlap_ratio = mid
            low = mid + 1e-6
        else:
            # 列数が少ない場合、範囲を下げる
            best_overlap_ratio = mid
            high = mid - 1e-6    
    df_results = df_filtered[df_filtered["overlap_ratio"] >= best_overlap_ratio]
    G = nx.from_pandas_edgelist(df_results, "marker1", "marker2")
    # ノードAと直接つながっているノードのみを取得
    neighbors = list(G.neighbors(gene_symbol))
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)
else:
    df_results = df_filtered

# nodesを用意
node_json = []
for node in subgraph.nodes():
    annotation = marker_mp_dict[node]
    node_color = 1 if node == gene_symbol else 0
    node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})
# 0.0s

In [136]:
P(len(subgraph.nodes)) # 91
PP(node_json[0])

20
{'data': {'annotation': ['abnormal behavior (Homo)',
                         'abnormal cholesterol homeostasis (Homo, ♀)',
                         'abnormal coat appearance (Hetero)',
                         'abnormal coat appearance (Homo)',
                         'abnormal freezing behavior (Homo)',
                         'abnormal inspiratory capacity (Homo)',
                         'abnormal kidney morphology (Homo)',
                         'abnormal lens morphology (Homo)',
                         'abnormal retina inner nuclear layer morphology '
                         '(Homo)',
                         'abnormal sinus arrhythmia (Homo)',
                         'abnormal sleep behavior (Hetero, ♀)',
                         'abnormal sleep behavior (Homo)',
                         'abnormal sleep behavior (Homo, ♀)',
                         'abnormal sleep behavior (Homo, ♂)',
                         'abnormal spleen morphology (Homo)',
                      

In [137]:
# edgesを用意
df_edge = df_overlap[
        (df_overlap['marker1'].isin(subgraph.nodes())) &
        (df_overlap['marker2'].isin(subgraph.nodes()))
    ]

edge_json = []
for edge in df_edge.itertuples():
    edge_json.append(
        {
            "data": {
                "source": edge.marker1,
                "target": edge.marker2,
                "edge_size": edge.overlap_ratio,
                "annotation": edge.overlapped_mp,
            }
        }
    )
P(len(edge_json)) # 91

95


In [138]:
edge_json[10]

{'data': {'source': 'Ap4e1',
  'target': 'Rhd',
  'edge_size': 0.057,
  'annotation': ['decreased circulating free fatty acids level (Homo)',
   'decreased mean corpuscular hemoglobin (Homo)',
   'decreased mean corpuscular hemoglobin concentration (Homo)',
   'decreased mean corpuscular volume (Homo)',
   'increased exploration in new environment (Homo)']}}

In [139]:
df_edge = df_overlap[
        (df_overlap['marker1'].isin(subgraph.nodes())) &
        (df_overlap['marker2'].isin(subgraph.nodes()))
    ]

In [140]:
df_edge[df_edge["marker1"] == "Anapc7"]


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp


In [141]:
df_edge[df_edge["marker2"] == "Anapc7"]


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp


In [142]:
# 昔の処理

G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")
df_overlap = df_overlap.copy()
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)
if len(subgraph.nodes) <= 500:
    df_filtered = df_overlap
else:
    target_columns = 500
    # 二分探索の範囲
    low, high = df_overlap["overlap_ratio"].min(), df_overlap["overlap_ratio"].max()
    best_overlap_ratio = None

    while low <= high:
        mid = (low + high) / 2

        # overlap_ratio >= mid のデータをフィルタリング
        df_mid = df_overlap[df_overlap["overlap_ratio"] >= mid]

        G = nx.from_pandas_edgelist(df_mid, "marker1", "marker2")
        # ノードAと直接つながっているノードのみを取得
        try:
            neighbors = list(G.neighbors(gene_symbol))
        except:
            high = mid - 1e-6
            continue        
        subgraph_nodes = [gene_symbol] + neighbors
        subgraph = G.subgraph(subgraph_nodes)

        row_count = len(subgraph.nodes)
        # ターゲット列数に近い場合、結果を保存
        if row_count == target_columns:
            best_overlap_ratio = mid
            break
        elif row_count > target_columns:
            # 列数が多い場合、範囲を上げる
            best_overlap_ratio = mid
            low = mid + 1e-6
        else:
            # 列数が少ない場合、範囲を下げる
            best_overlap_ratio = mid
            high = mid - 1e-6

    df_filtered = df_overlap[df_overlap["overlap_ratio"] >= best_overlap_ratio]

# 前処理
G = nx.from_pandas_edgelist(df_filtered, "marker1", "marker2")
df_filtered = df_filtered.copy()
df_filtered["id"] = df_filtered.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)
# nodesを用意
node_json = []
for node in subgraph.nodes():
    annotation = marker_mp_dict[node]
    node_color = 1 if node == gene_symbol else 0
    node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

# 5.8s

In [143]:
P(len(subgraph.nodes)) # 91
PP(node_json[:3])

20
[{'data': {'annotation': ['abnormal behavior (Homo)',
                          'abnormal cholesterol homeostasis (Homo, ♀)',
                          'abnormal coat appearance (Hetero)',
                          'abnormal coat appearance (Homo)',
                          'abnormal freezing behavior (Homo)',
                          'abnormal inspiratory capacity (Homo)',
                          'abnormal kidney morphology (Homo)',
                          'abnormal lens morphology (Homo)',
                          'abnormal retina inner nuclear layer morphology '
                          '(Homo)',
                          'abnormal sinus arrhythmia (Homo)',
                          'abnormal sleep behavior (Hetero, ♀)',
                          'abnormal sleep behavior (Homo)',
                          'abnormal sleep behavior (Homo, ♀)',
                          'abnormal sleep behavior (Homo, ♂)',
                          'abnormal spleen morphology (Homo)',
      

In [144]:
old_nodes = {s for s in subgraph.nodes()}

In [145]:
new_nodes - old_nodes

{'Adgrf5',
 'Adgrl4',
 'Adipor1',
 'Anapc7',
 'Ankrd13c',
 'Ap2a1',
 'Atad2b',
 'Baat',
 'Bhlhe40',
 'C2cd5',
 'Cdkn1b',
 'Clcnkb',
 'Cracdl',
 'Dbnl',
 'Dcun1d3',
 'Dnajc10',
 'Dnajc7',
 'Dym',
 'Ecrg4',
 'Enc1',
 'Exd1',
 'Fam135a',
 'Far2',
 'Fbf1',
 'Fbxo10',
 'Gc',
 'Gda',
 'Gimap6',
 'Hemgn',
 'Hsd17b1',
 'Hspb6',
 'Idh2',
 'Kctd1',
 'Lactb',
 'Lpin2',
 'Mmp11',
 'Mtmr3',
 'Mysm1',
 'Nab2',
 'Necap2',
 'Ntaq1',
 'Osm',
 'Pcdhb10',
 'Peak1',
 'Pex3',
 'Pgam5',
 'R3hcc1l',
 'Rab11b',
 'Rab3il1',
 'Rrbp1',
 'Sec24a',
 'Selenbp1',
 'Sidt2',
 'Sik3',
 'Slc25a21',
 'Sox21',
 'Sqstm1',
 'Steap3',
 'Stx2',
 'Tbc1d9b',
 'Tmem203',
 'Tpcn1',
 'Trim56',
 'Ttc28',
 'Txnl1',
 'Vil1',
 'Wdr27',
 'Wdr41',
 'Zfand6',
 'Zfp84',
 'Zzef1'}